In [ ]:
# !pip install torch transformers

In [ ]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer
import os

In [ ]:
import requests
import json
name = 'tokenizer.json'
url = f'https://huggingface.co/bert-large-uncased-whole-word-masking-finetuned-squad/raw/main/{name}'
resp = requests.get(url, verify=False)
dict_ = resp.json()
json_object = json.dumps(dict_, indent = 4)


url = 'https://huggingface.co/bert-large-uncased-whole-word-masking-finetuned-squad/raw/main/vocab.txt'
resp = requests.get(url,verify=False)
dict_ = resp.text


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097

In [ ]:
import torch
from transformers import BertForQuestionAnswering, BertTokenizer

def find_answer_text(question, text):
    model_name = 'bert-large-uncased-whole-word-masking-finetuned-squad'
    model = BertForQuestionAnswering.from_pretrained(model_name)
    tokenizer = BertTokenizer.from_pretrained(model_name)

    input_ids = tokenizer.encode(question, text)
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Locate the first [SEP] token
    sep_idx = input_ids.index(tokenizer.sep_token_id)
    num_seg_a = sep_idx + 1  # Tokens in segment A (question)
    num_seg_b = len(input_ids) - num_seg_a  # Tokens in segment B (text)
    segment_ids = [0] * num_seg_a + [1] * num_seg_b  # Segment identifiers

    assert len(segment_ids) == len(input_ids)

    # Model inference
    output = model(
        torch.tensor([input_ids]),
        token_type_ids=torch.tensor([segment_ids])
    )

    # Extract answer
    answer_start = torch.argmax(output.start_logits)
    answer_end = torch.argmax(output.end_logits)

    if answer_end >= answer_start:
        answer = " ".join(tokens[answer_start:answer_end + 1])
    else:
        return "I am unable to find the answer to this question. Can you please ask another question?"

    formatted_answer = answer.capitalize()
    print(f"\nQuestion:\n{question.capitalize()}")
    print(f"\nAnswer:\n{formatted_answer}.")

    return formatted_answer

In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.7 MB/s eta 0:00:00


In [ ]:
# import pdfplumber

# pdf_path = "paper.pdf"

# full_text = ""
# with pdfplumber.open(pdf_path) as pdf:a
#     for page in pdf.pages:
#         text = page.extract_text()
#         if text:  # Avoid None values
#             full_text += text + "\n"

# print(full_text)  # Print or process the extracted text


In [ ]:
question = 'What are the problems the authors are solving?'
question = 'What solution is presented?'
# question = 'What are the limitations of the solution?'
text = '''KOMPSAT-3 is a high performance remote sensing satellite, which provides 0.7 m GSD
panchromatic image and 2.8 m GSD multi-spectral image data for various applications.
KOMPSAT-3 was launched into a sun synchronous low Earth orbit on the 18th of May, 2012
and the life time of more than 7 years is expected.'''
text2 = '''Forecasting high-impact research topics
via machine learning on evolving knowledge graphs
Xuemei Gu1, ∗ and Mario Krenn1, †
1Max Planck Institute for the Science of Light, Staudtstrasse 2, 91058 Erlangen, Germany
The exponential growth in scientific publications poses a severe challenge for human researchers.
It forces attention to more narrow sub-fields, which makes it challenging to discover new impactful
research ideas and collaborations outside one’s own field. While there are ways to predict a scientific
paper’s future citation counts, they need the research to be finished and the paper written, usually
assessing impact long after the idea was conceived. Here we show how to predict the impact of
onsets of ideas that have never been published by researchers. For that, we developed a large
evolving knowledge graph built from more than 21 million scientific papers. It combines a semantic
network created from the content of the papers and an impact network created from the historic
citations of papers. Using machine learning, we can predict the dynamic of the evolving network
into the future with high accuracy (AUC values beyond 0.9 for most experiments), and thereby the
impact of new research directions. We envision that the ability to predict the impact of new ideas
will be a crucial component of future artificial muses that can inspire new impactful and interesting
scientific ideas.
'''

answer = find_answer_text(question,text2)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Question:
What solution is presented?

Answer:
Using machine learning , we can predict the dynamic of the evolving network into the future with high accuracy ( au ##c values beyond 0 . 9 for most experiments ) , and thereby the impact of new research directions.


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Replace with the actual model name (it should be a Qwen model with < 3B parameters)
model_name = "Qwen/Qwen2.5-0.5B"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Load the model and move it to GPU if available
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")


def extract_info(abstract: str) -> dict:
    # Construct a prompt that instructs the model on what to extract
    prompt = (
        f'''
        Extract from the article text the following information:\n
        - Problem:\n
        - Solution:\n
        - Limitation:\n
        return a concise answer with the same format as the prompt.
        article text: {abstract}\n\n
        '''
    )

    # Encode prompt and generate output
    inputs = {k: v.to("cuda") for k, v in tokenizer(prompt, return_tensors="pt").items()}

    outputs = model.generate(**inputs, max_length=512, num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_text)
    # Initialize the result dictionary
    result = {"problem": None, "solution": None, "limitation": None}

    # Parse the output to extract each section by looking for markers
    for line in generated_text.splitlines():
        line = line.strip()
        if line.lower().startswith("problem:"):
            result["problem"] = line[len("problem:"):].strip()
        elif line.lower().startswith("solution:"):
            result["solution"] = line[len("solution:"):].strip()
        elif line.lower().startswith("limitation:"):
            result["limitation"] = line[len("limitation:"):].strip()

    return result

# Example usage:
if __name__ == "__main__":
    abstract_text = (
        "In this paper, we address the challenge of efficiently training neural networks on limited data. "
        "Our solution introduces a novel data augmentation technique combined with transfer learning. "
        "However, our approach is limited by the increased computational overhead in the augmentation step."
    )

    info = extract_info(abstract_text)
    print("Extracted Information:")
    print("Problem    :", info["problem"])
    print("Solution   :", info["solution"])
    print("Limitation :", info["limitation"])


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



        Extract from the article text the following information:

        - Problem:

        - Solution:

        - Limitation:

        return a concise answer with the same format as the prompt.
        article text: In this paper, we address the challenge of efficiently training neural networks on limited data. Our solution introduces a novel data augmentation technique combined with transfer learning. However, our approach is limited by the increased computational overhead in the augmentation step.


        


        """
        # Write your code here
        # Hint: You can use the following code to get the number of words in the article
        # words = re.findall(r'\w+', article_text)
        # print(len(words))
        # Hint: You can use the following code to get the number of sentences in the article
        # sentences = re.findall(r'(?<=\s)\w+', article_text)
        # print(len(sentences))
        # Hint: You can use the following code to get the number of paragraphs 